In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [2]:
pd.set_option('display.max_columns', None)

## Global Food Prices Database (Cleaned)
Updated Global Market Food Prices with 1.048M rows

In [3]:
price_df = pd.read_csv('../raw_data/Global Food Prices Database/wfp_food_prices_database.csv')

/Users/plyck/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [5]:
price_df.isnull().sum()/len(price_df)

adm0_id               0.000000
adm0_name             0.000000
adm1_id               0.000000
adm1_name             0.302027
mkt_id                0.000000
mkt_name              0.000000
cm_id                 0.000000
cm_name               0.000000
cur_id                0.000000
cur_name              0.000000
pt_id                 0.000000
pt_name               0.000000
um_id                 0.000000
um_name               0.000000
mp_month              0.000000
mp_year               0.000000
mp_price              0.000000
mp_commoditysource    1.000000
dtype: float64

In [6]:
#dropping the mp_commoditysource because it's 100% Nan's

price_df.drop(columns='mp_commoditysource', inplace=True)

In [7]:
price_df['adm1_name'].nunique()

613

In [8]:
price_df['adm1_name'].fillna("Unknown", inplace=True)

In [9]:
price_df['adm1_name'].nunique()

614

In [10]:
price_df.isnull().sum()/len(price_df)

adm0_id      0.0
adm0_name    0.0
adm1_id      0.0
adm1_name    0.0
mkt_id       0.0
mkt_name     0.0
cm_id        0.0
cm_name      0.0
cur_id       0.0
cur_name     0.0
pt_id        0.0
pt_name      0.0
um_id        0.0
um_name      0.0
mp_month     0.0
mp_year      0.0
mp_price     0.0
dtype: float64

In [11]:
# Introducing new column 'type' where the cm_name (commodity name) is split to make it easier to read by the nutrition API

In [12]:
price_df['type'] = price_df['cm_name'].apply(lambda x: x.split('-')[0])

In [13]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,type
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,Bread
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,Bread
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,Bread
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,Bread
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,Bread


In [14]:
price_2020_df = price_df[price_df['mp_year'] == 2020]

In [15]:
price_2020_df.shape

(165098, 18)

In [16]:
price_2020_df['cur_name'].unique()

array(['AFN', 'AOA', 'ARS', 'AMD', 'BDT', 'INR', 'BYR', 'XOF', 'BTN',
       'BOB', 'BIF', 'KHR', 'XAF', 'CVE', 'CNY', 'COP', 'CDF', 'DJF',
       'DOP', 'USD', 'EGP', 'ETB', 'GMD', 'GHS', 'GTQ', 'GNF', 'HTG',
       'HNL', 'IDR', 'IRR', 'IQD', 'JPY', 'JOD', 'KZT', 'KES', 'KGS',
       'LAK', 'LBP', 'LSL', 'LRD', 'LYD', 'MWK', 'SOS', 'MRO', 'MXN',
       'MNT', 'MZN', 'MMK', 'NAD', 'NPR', 'NIO', 'NGN', 'PKR', 'PAB',
       'PYG', 'PEN', 'PHP', 'RUB', 'RWF', 'SLL', 'ZAR', 'SSP', 'LKR',
       'NIS', 'SDG', 'SZL', 'SYP', 'TJS', 'THB', 'TRY', 'UGX', 'TZS',
       'YER', 'ZMW'], dtype=object)

In [17]:
# Source Wiki: It was previously known as the new Israeli shekel and the non-official abbreviation of NIS.
# Need to change NIS to ILS (Israeli New Shekel)

In [18]:
price_2020_df['cur_name'] = price_2020_df['cur_name'].apply(lambda x: x.replace('NIS','ILS'))

/var/folders/ld/8g4lbtw14x12q9rxlrk7ccx00000gn/T/ipykernel_1141/2113586509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_2020_df['cur_name'] = price_2020_df['cur_name'].apply(lambda x: x.replace('NIS','ILS'))


In [19]:
currencies = price_2020_df['cur_name'].unique()

In [20]:
type(currencies)

numpy.ndarray

In [21]:
currencies = currencies.tolist()

In [22]:
currencies.sort()

In [23]:
len(currencies)

74

In [24]:
#SSP = South Sudanese pound
# This is not in our currency exchange API so must be removed and added later.

In [25]:
currencies.remove("SSP")

In [26]:
len(currencies)

73

In [27]:
#API call to get the conversion rates in the list currencies

In [28]:
api_key = 'e3479b3ac491d8699be3a12b7cf7e51d'

In [29]:
curr = ','.join(currencies)

In [30]:
response = requests.get(f'http://api.currencylayer.com/live?access_key={api_key}&currencies={curr}&format=1')

In [31]:
response.url

'http://api.currencylayer.com/live?access_key=e3479b3ac491d8699be3a12b7cf7e51d&currencies=AFN,AMD,AOA,ARS,BDT,BIF,BOB,BTN,BYR,CDF,CNY,COP,CVE,DJF,DOP,EGP,ETB,GHS,GMD,GNF,GTQ,HNL,HTG,IDR,ILS,INR,IQD,IRR,JOD,JPY,KES,KGS,KHR,KZT,LAK,LBP,LKR,LRD,LSL,LYD,MMK,MNT,MRO,MWK,MXN,MZN,NAD,NGN,NIO,NPR,PAB,PEN,PHP,PKR,PYG,RUB,RWF,SDG,SLL,SOS,SYP,SZL,THB,TJS,TRY,TZS,UGX,USD,XAF,XOF,YER,ZAR,ZMW&format=1'

In [32]:
response.json()

{'success': True,
 'terms': 'https://currencylayer.com/terms',
 'privacy': 'https://currencylayer.com/privacy',
 'timestamp': 1638951004,
 'source': 'USD',
 'quotes': {'USDAFN': 96.25256,
  'USDAMD': 491.785255,
  'USDAOA': 564.209003,
  'USDARS': 101.364698,
  'USDBDT': 85.609632,
  'USDBIF': 1988.457958,
  'USDBOB': 6.88221,
  'USDBTN': 75.266214,
  'USDBYR': 19600,
  'USDCDF': 2001.99979,
  'USDCNY': 6.355302,
  'USDCOP': 3910.98,
  'USDCVE': 97.76566,
  'USDDJF': 177.677891,
  'USDDOP': 56.509482,
  'USDEGP': 15.729398,
  'USDETB': 48.065551,
  'USDGHS': 6.113668,
  'USDGMD': 52.50026,
  'USDGNF': 9482.344389,
  'USDGTQ': 7.715654,
  'USDHNL': 24.109684,
  'USDHTG': 100.802269,
  'USDIDR': 14360.45,
  'USDILS': 3.11779,
  'USDINR': 75.539018,
  'USDIQD': 1456.183202,
  'USDIRR': 42249.999669,
  'USDJOD': 0.709198,
  'USDJPY': 113.4265,
  'USDKES': 112.839953,
  'USDKGS': 84.797101,
  'USDKHR': 4063.738309,
  'USDKZT': 436.408126,
  'USDLAK': 10905.995939,
  'USDLBP': 1509.30014,
  

In [33]:
conversion_rates = response.json()

In [34]:
conversion_rates = conversion_rates['quotes']

In [35]:
len(conversion_rates), len(currencies)

(73, 73)

In [36]:
# Making a dictionary of the conversion rates

In [37]:
dic_values = []
for x in conversion_rates.keys():
    dic_values.append(conversion_rates[x])

In [38]:
len(dic_values)

73

In [39]:
conversion_rates_dict = dict(zip(currencies,dic_values))

In [40]:
#SSP will be added as 1 USD = 130.26 SSP

In [41]:
conversion_rates_dict['SSP'] = 130.26

In [42]:
# Adding a new column to our data with the conversion rates

In [43]:
price_2020_df['conv_rate'] = list(map(lambda x: conversion_rates_dict[x] , price_2020_df['cur_name']))

/var/folders/ld/8g4lbtw14x12q9rxlrk7ccx00000gn/T/ipykernel_1141/4267227431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_2020_df['conv_rate'] = list(map(lambda x: conversion_rates_dict[x] , price_2020_df['cur_name']))


In [44]:
price_2020_df['conv_rate'].nunique()

74

In [45]:
# Adding a new column to our data with the price in USD

In [46]:
price_2020_df["usd_rate"] = price_2020_df["mp_price"] / price_2020_df["conv_rate"] 

/var/folders/ld/8g4lbtw14x12q9rxlrk7ccx00000gn/T/ipykernel_1141/1950825033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_2020_df["usd_rate"] = price_2020_df["mp_price"] / price_2020_df["conv_rate"]


In [47]:
price_2020_df

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate
69,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2020,50.0000,Bread,96.252560,0.519467
70,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2020,50.0000,Bread,96.252560,0.519467
71,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2020,50.0000,Bread,96.252560,0.519467
72,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2020,50.0000,Bread,96.252560,0.519467
275,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,84,Wheat - Retail,0.0,AFN,15,Retail,5,KG,1,2020,25.3750,Wheat,96.252560,0.263629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556443,270.0,Zambia,3434,Western,1462,Kalabo,265,"Maize meal (white, breakfast) - Retail",0.0,ZMW,15,Retail,5,KG,2,2020,6.4480,"Maize meal (white, breakfast)",17.416263,0.370229
1556444,270.0,Zambia,3434,Western,1462,Kalabo,265,"Maize meal (white, breakfast) - Retail",0.0,ZMW,15,Retail,5,KG,3,2020,6.6320,"Maize meal (white, breakfast)",17.416263,0.380794
1556445,270.0,Zambia,3434,Western,1462,Kalabo,265,"Maize meal (white, breakfast) - Retail",0.0,ZMW,15,Retail,5,KG,4,2020,6.4400,"Maize meal (white, breakfast)",17.416263,0.369769
1556446,270.0,Zambia,3434,Western,1462,Kalabo,265,"Maize meal (white, breakfast) - Retail",0.0,ZMW,15,Retail,5,KG,5,2020,6.5168,"Maize meal (white, breakfast)",17.416263,0.374179


In [48]:
price_2020_df.shape

(165098, 20)

In [49]:
price_2020_df['mp_month'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

## Importing Adrians Dataset

In [50]:
food_types = pd.read_csv('../raw_data/Food Types/food_types_v1.csv')

In [51]:
food_types.head()

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
1,Wheat,10.7,4.53,47.5,5.73,473,0.0,125,274
2,Rice (low quality),0.3,19.2,16.0,4.2,110,24.0,0.0,967
3,Wage (qualified labour),0.0,0.0,71.4,not found,5360.0,not found,not found,357
4,Fuel (diesel),0.84,0.0,8.86,7.59,36.0,not found,15.0,38.0


In [52]:
food_types['food_type'].nunique(), price_2020_df['type'].nunique()

(522, 522)

In [53]:
price_2020_df.shape

(165098, 20)

In [56]:
joined_df = pd.merge(price_2020_df, food_types, how='outer', left_on = 'type', right_on = 'food_type')

In [57]:
joined_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
4,1.0,Afghanistan,272,Badakhshan,267,Mazar,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2020,60.0,Bread,96.25256,0.623360,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306


In [76]:
joined_df['mp_year'].unique()

array([2020])

# Drop columns with XXX_id as they do not mean much for us

In [78]:
df_condensed = joined_df.drop(columns=['adm0_id', 'adm1_id', 'mkt_id', 'cm_id', 'cur_id', 'pt_id', 'um_id'])

In [79]:
df_condensed.head()

,adm0_name,adm1_name,mkt_name,cm_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,1,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
1,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,2,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
2,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,3,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
3,Afghanistan,Badakhshan,Fayzabad,Bread - Retail,AFN,Retail,KG,4,2020,50.0,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
4,Afghanistan,Badakhshan,Mazar,Bread - Retail,AFN,Retail,KG,1,2020,60.0,Bread,96.25256,0.623360,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
